<a href="https://colab.research.google.com/github/i-SanMartin/FastAI/blob/main/04_MNIST_Basics/Network_Exercise_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook fastai==2.2.5
import fastbook

In [2]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
path.ls()

(#2) [Path('/root/.fastai/data/mnist_png/training'),Path('/root/.fastai/data/mnist_png/testing')]

In [5]:
Path.BASE_PATH = path

In [6]:
(path/'training').ls()

(#10) [Path('training/2'),Path('training/5'),Path('training/8'),Path('training/7'),Path('training/6'),Path('training/0'),Path('training/9'),Path('training/3'),Path('training/4'),Path('training/1')]

In [7]:
categories = tuple(x for x in range(0,10))
categories

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [8]:
train_x = torch.tensor([])
train_y = torch.tensor([])
valid_x = torch.tensor([])
valid_y = torch.tensor([])

In [9]:
for x in categories:
  number_imgs = (path/'training'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  train_x = torch.cat([train_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  train_y = torch.cat([train_y,y])

In [10]:
for x in categories:
  number_imgs = (path/'testing'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  valid_x = torch.cat([valid_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  valid_y = torch.cat([valid_y,y])

In [11]:
train_x = train_x.view(-1,28*28).float()/255
valid_x = valid_x.view(-1,28*28).float()/255
train_y = train_y.unsqueeze(1).long()
valid_y = valid_y.unsqueeze(1).long()
train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 1]),
 torch.Size([10000, 784]),
 torch.Size([10000, 1]))

In [12]:
dset = list(zip(train_x,train_y))
dl = DataLoader(dset, batch_size=256)
valid_dset = list(zip(valid_x,valid_y))
valid_dl = DataLoader(dset, batch_size=256)
dls = DataLoaders(dl, valid_dl)

In [24]:
class optimizer():
  def __init__(self, parameters, lr):
    self.parameters = list(parameters)
    self.lr = lr
  def step(self):
    for p in self.parameters:
      p.data -= p.grad
  def grads_zero(self):
    for p in self.parameters:
      p.grad = None
  def calc_grad(self,model,criterion,x,y):
    pred = model(x)
    loss = criterion(pred,y.squeeze(1))
    loss.backward()

In [14]:
dls.loaders[0]

In [ ]:
simple_net = nn.Sequential(
    nn.Linear(28*28,200),
    nn.ReLU(),
    nn.Linear(200,100),
    nn.ReLU(),
    nn.Linear(100,30),
    nn.ReLU(),
    nn.Linear(30,len(categories)),
    nn.Sigmoid()
)

In [45]:
def batch_accuracy(x,y):
    pred = simple_net(x)

    pred_class = torch.tensor([torch.argmax(i) for i in pred])
    correct = (pred_class == y.squeeze(1)).float().sum()
    return correct

In [55]:
class learner():
  def __init__(self, Dls, model, optimize, loss_func, metrics):
    self.dls = Dls
    self.model = model
    self.opt = optimize
    self.loss_func = loss_func
    self.metrics = metrics

  def train(self, epochs, lr):
    self.Opt = self.opt(self.model.parameters(), lr)
    metric = 0
    for i in range(epochs):
      self.train_epoch(dl)
      metric = self.validate_epoch(valid_dl)
      loss = self.loss_func(self.model(valid_x),valid_y.squeeze(1))
      print("Epoch: {}   Metric:  {}   Loss:   {}".format(i, metric, loss))

  def train_epoch(self, data):
    for dx, dy in data:
      self.Opt.calc_grad(self.model,self.loss_func,dx,dy)
      self.Opt.step()
      self.Opt.grads_zero()

  def validate_epoch(self, data):
    accs = torch.tensor([0.])
    for xb,yb in data:
      accs += self.metrics(xb,yb)
    return accs/len(data)
  

In [53]:
learn = learner(dls, simple_net, optimizer, nn.CrossEntropyLoss(), batch_accuracy)

In [54]:
learn.train(50, 1)

Epoch: 0   Metric:  tensor([28.6894])   Loss:   2.3804633617401123
Epoch: 1   Metric:  tensor([28.6894])   Loss:   2.3804633617401123
Epoch: 2   Metric:  tensor([28.6894])   Loss:   2.3804633617401123
Epoch: 3   Metric:  tensor([28.6894])   Loss:   2.3804633617401123
Epoch: 4   Metric:  tensor([28.6894])   Loss:   2.3804633617401123
Epoch: 5   Metric:  tensor([28.6894])   Loss:   2.3804633617401123
Epoch: 6   Metric:  tensor([28.6894])   Loss:   2.3804633617401123
Epoch: 7   Metric:  tensor([28.6894])   Loss:   2.3804633617401123
Epoch: 8   Metric:  tensor([28.6894])   Loss:   2.3804633617401123
Epoch: 9   Metric:  tensor([28.6894])   Loss:   2.3804633617401123


KeyboardInterrupt: ignored